In [1]:
import pandas as pd
import sklearn

In [2]:
df = pd.read_csv('LA pm25.csv', parse_dates=['date', 'utc','local'], index_col='utc')

In [3]:
df.head()

,parameter,location,value,concentration_units,city,country,sourcename,sourcetype,mobile,latitude,...,stp,visibility,wind_speed,max_sustained_wind,gust,max_temp,min_temp,precipitation,snow_depth,weather_events
utc,,,,,,,,,,,,,,,,,,,,,
2020-01-02 16:00:00,pm25,North Holywood,12.9,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 22:00:00,pm25,North Holywood,13.5,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 17:00:00,pm25,North Holywood,14.4,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 19:00:00,pm25,North Holywood,9.1,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN
2020-01-02 18:00:00,pm25,North Holywood,11.0,µg/m³,Los Angeles-Long Beach-Santa Ana,US,AirNow,government,False,34.181976,...,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0,NaN


In [4]:
df.columns

Index(['parameter', 'location', 'value', 'concentration_units', 'city',
       'country', 'sourcename', 'sourcetype', 'mobile', 'latitude',
       'longitude', 'local', 'source', 'url', 'averaging_time_unit',
       'averaging_time', 'station', 'date', 'latitude-2', 'longitude-2',
       'elevation', 'name', 'temperature', 'dewpoint', 'slp', 'stp',
       'visibility', 'wind_speed', 'max_sustained_wind', 'gust', 'max_temp',
       'min_temp', 'precipitation', 'snow_depth', 'weather_events'],
      dtype='object')

In [5]:
df.value.isna().sum()

0

In [6]:
df = df.resample('d').mean()
df = df.interpolate(method='time')
df.head()

,value,mobile,latitude,longitude,url,averaging_time,station,latitude-2,longitude-2,elevation,...,slp,stp,visibility,wind_speed,max_sustained_wind,gust,max_temp,min_temp,precipitation,snow_depth
utc,,,,,,,,,,,,,,,,,,,,,
2020-01-02,11.355556,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1012.5,986.8,10.0,4.5,12.0,NaN,68.0,43.0,0.0,0.0
2020-01-03,18.295833,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1019.9,994.0,10.0,1.6,6.0,NaN,73.0,45.0,0.0,0.0
2020-01-04,21.345833,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1025.1,999.1,10.0,1.1,6.0,NaN,73.0,42.1,0.0,0.0
2020-01-05,20.958333,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1024.2,998.0,10.0,2.8,11.1,18.100000,73.0,42.1,0.0,0.0
2020-01-06,11.992857,0.0,34.181976,-118.36304,NaN,1.0,7.228802e+10,34.20056,-118.3575,225.9,...,1023.5,997.6,10.0,2.5,6.0,16.733333,75.0,42.1,0.0,0.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print(f'Input: Height, age')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Input: Height, age
    Default Settings
Internal Parameters:
   Bias is 264.5094346781485
   Coefficients: [-0.2535813  -0.62651179 -0.35701325  0.44027779 -0.16480394 -0.37586064
  0.14215903]
   Score 0.19438899673401866
MAE is 3.870967987634164
RMSE is 6.8867459752794105
MSE is 47.42727012802716
R^2: 0.19438899673401866


In [8]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor

power = 2
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is -6548.73182767346
   Coefficients: [-1.42549783e-09  1.34616338e+01 -1.16108084e+02  5.50842222e+01
 -4.20581372e+01 -2.50570675e+01  7.19401897e+01  1.68909977e+00
 -6.90752301e-03  1.13787856e-01 -5.53662854e-02  4.29475066e-02
  2.48745405e-02 -7.27414998e-02 -1.26504147e-03  1.56951642e-01
 -2.52946454e-02 -1.55325860e-02  9.92949600e-02 -5.24640310e-02
  1.66289260e-02  1.84652757e-02  4.97964420e-03 -5.03400328e-02
  2.84699550e-02 -1.78639663e-02  1.70908008e-03  1.04341288e-02
 -1.91914377e-02  1.04913254e-02 -3.30695847e-02  5.10576726e-02
 -5.07671952e-03 -8.68064878e-03 -1.23625727e-02  1.24597682e-03]
   Score 0.25207746914536533
MAE is 3.742200289460802
RMSE is 6.635592038623982
MSE is 44.03108170304998
R^2: 0.25207746914536533


In [9]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 2
MAE is 3.5712581540857258
RMSE is 5.0570149951620325
MSE is 25.573400661293654
R^2: 0.30576109397254647


In [10]:
power = 3
poly_process = PolynomialFeatures(degree = power, include_bias=True)

train_set, test_set = train_test_split(df, test_size = 0.2, random_state = 123)
X = train_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 58073.99760086355
   Coefficients: [ 2.59835785e-02 -6.23060234e+01 -9.99569335e+03  5.55161897e+03
 -1.75065966e+03  4.34929190e+03 -4.04027344e+03  7.75225709e+02
 -5.69447575e-02  1.96639427e+01 -1.09269821e+01  2.77092453e+00
 -9.29812081e+00  9.66977431e+00 -1.55516354e+00  9.73200890e+01
 -5.36769792e+01 -2.85909531e+01 -3.18010491e+00  3.67351041e+01
 -6.26855816e-01  6.53650291e+00  3.67671114e+01  3.03498255e+01
 -7.18075060e+01 -8.14956308e-01  3.34546866e+00  5.43959661e+00
 -1.04068266e+01  6.34666417e+00  1.04142443e+01 -1.48134227e+01
 -8.50014221e+00  6.63623293e+00 -6.79859050e-01 -2.58973469e-01
  6.10483048e-05 -9.66180217e-03  5.38033452e-03 -1.01196357e-03
  4.95413698e-03 -5.63799804e-03  7.80566371e-04 -9.51848382e-02
  5.25202522e-02  3.04540917e-02  4.29251362e-03 -4.01377854e-02
  4.66009004e-04 -6.46256933e-03 -3.75442442e-02 -3.14845999e-02
  7.35702394e-02  7.40325345e-04 -3.25720

In [11]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 3
MAE is 3.956253089208536
RMSE is 5.700748370958527
MSE is 32.4985319889863
R^2: 0.11776515003418608


In [12]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=1000, max_features = 'sqrt', max_depth =100,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')


MAE is 1.4018854980585693
RMSE is 2.5068236552692245
MSE is 6.284164838617356
R^2: 0.8932556664834144


In [13]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 3
MAE is 3.6804231382569066
RMSE is 5.280020497618539
MSE is 27.878616455271924
R^2: 0.24318159927941774


In [14]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=100, max_features = 'sqrt', max_depth =10,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 2.200154322282721
RMSE is 3.419097890387967
MSE is 11.690230384055448
R^2: 0.8014269384957932


In [15]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for RF Regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for RF Regression on test data
MAE is 3.6215856217149804
RMSE is 5.249501202529769
MSE is 27.5572628753615
R^2: 0.2519053572465263


In [16]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=10, max_features = 'sqrt', max_depth =10,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 2.350966556921341
RMSE is 3.74756321604158
MSE is 14.04423005822791
R^2: 0.7614413345578346


In [17]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for RF Regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for RF Regression on test data
MAE is 3.9723274876456895
RMSE is 5.62952028871595
MSE is 31.691498681064516
R^2: 0.1396736137633503


In [18]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=50, max_features = 'sqrt', max_depth =5,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 3.4390888810268336
RMSE is 5.729108304332828
MSE is 32.82268196277538
R^2: 0.4424660395900457


In [19]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for RF Regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for RF Regression on test data
MAE is 3.740823812044581
RMSE is 5.3375741459305015
MSE is 28.489697763305724
R^2: 0.2265926276208573


In [20]:
from sklearn.ensemble import RandomForestRegressor
rfmodel = RandomForestRegressor(n_estimators=30, max_features = 'sqrt', max_depth =5,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 3.444619329153732
RMSE is 5.839383908311123
MSE is 34.09840442864289
R^2: 0.4207963113336616


In [21]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for RF Regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for RF Regression on test data
MAE is 3.7413219054574225
RMSE is 5.385351396285301
MSE is 29.002009661472034
R^2: 0.21268494062848387


In [22]:
rfmodel = RandomForestRegressor(n_estimators=30, max_features = 'sqrt', max_depth =7,random_state =123)
rfmodel.fit(X,y)

y_pred = rfmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 2.974656013751278
RMSE is 4.775091678937848
MSE is 22.80150054226148
R^2: 0.6126882344643776


In [23]:
y_test_pred = rfmodel.predict(X_test)
print()
print(f'Results for RF Regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for RF Regression on test data
MAE is 3.7817339931164438
RMSE is 5.443952796270193
MSE is 29.636622048018058
R^2: 0.19545717280744446


In [24]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

mlpmodel = MLPRegressor(hidden_layer_sizes=(100,400,100), activation='tanh', random_state =123, max_iter=2000)
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X)
X_testscaled=sc_X.transform(X_test)
mlpmodel.fit(X_trainscaled,y)
y_pred = mlpmodel.predict(X_trainscaled)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 1.0127762561457585
RMSE is 3.2873119640844033
MSE is 10.806419949212458
R^2: 0.8164395548489752


In [25]:
y_test_pred = mlpmodel.predict(X_testscaled)
print()
print(f'Results for linear regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
MAE is 4.404028577490292
RMSE is 6.011301639132784
MSE is 36.13574739664049
R^2: 0.019025976506205633


In [26]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

mlpmodel = MLPRegressor(hidden_layer_sizes=(25,50,25), activation='tanh', random_state =123, max_iter=2000)
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X)
X_testscaled=sc_X.transform(X_test)
mlpmodel.fit(X_trainscaled,y)
y_pred = mlpmodel.predict(X_trainscaled)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 1.76792330682285
RMSE is 3.876078190500755
MSE is 15.023982138875605
R^2: 0.7447990303621291


C:\Users\Matthew\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (2000) reached and the optimization hasn't converged yet.
  warnings.warn(


In [27]:
y_test_pred = mlpmodel.predict(X_testscaled)
print()
print(f'Results for linear regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
MAE is 4.0426676979987795
RMSE is 5.540617750259498
MSE is 30.698445054490623
R^2: 0.16663195506766948


In [28]:
from sklearn.neural_network import MLPRegressor

mlpmodel = MLPRegressor(hidden_layer_sizes=(25,50,25), activation='tanh', random_state =123, max_iter=2000)
mlpmodel.fit(X,y)

y_pred = mlpmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 3.9300632737915824
RMSE is 7.081135048698817
MSE is 50.1424735779108
R^2: 0.14826789869427792


In [29]:
y_test_pred = mlpmodel.predict(X_test)
print()
print(f'Results for model regression on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for model regression on test data
MAE is 3.7179776347116276
RMSE is 5.546062995064715
MSE is 30.758814745226196
R^2: 0.16499310427075864


In [30]:
from sklearn.neural_network import MLPRegressor

mlpmodel = MLPRegressor(hidden_layer_sizes=(50,100,50), activation='tanh', random_state =123, max_iter=2000)
mlpmodel.fit(X,y)

y_pred = mlpmodel.predict(X)
print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

MAE is 3.924224813909053
RMSE is 7.188351529699855
MSE is 51.672397714538235
R^2: 0.12228023979461655


In [31]:
y_test_pred = mlpmodel.predict(X_test)
print()
print(f'Results for model on test data')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for model on test data
MAE is 3.722428731640014
RMSE is 5.560672332773334
MSE is 30.92107679247083
R^2: 0.16058819044405015


In [32]:
X = train_set[['stp', 'wind_speed', 'max_temp' , 'dewpoint']]
y = train_set['value']

X_test = test_set[['stp', 'wind_speed', 'max_temp' , 'dewpoint']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 190.44153288310503
   Coefficients: [-0.18132066 -1.38173996  0.07675422  0.09147388]
   Score 0.15303437812371745
MAE is 4.106448872859112
RMSE is 7.061293458776801
MSE is 49.861865310964035
R^2: 0.15303437812371745


In [33]:
X = train_set[['wind_speed']]
y = train_set['value']

X_test = test_set[['wind_speed']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print(f'    Default Settings')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
    Default Settings
Internal Parameters:
   Bias is 20.371818944447657
   Coefficients: [-1.12450609]
   Score 0.0787488071227952
MAE is 4.435974274036218
RMSE is 7.364451405856905
MSE is 54.23514450922775
R^2: 0.0787488071227952


In [34]:
X = train_set[['max_sustained_wind']]
y = train_set['value']

X_test = test_set[['max_sustained_wind']]
y_test = test_set['value']

lrmodel = LinearRegression()
lrmodel.fit(X,y)

y_pred = lrmodel.predict(X)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 23.080062444272215
   Coefficients: [-0.75126272]
   Score 0.10246921830642874
MAE is 4.366056633351809
RMSE is 7.269023037539288
MSE is 52.83869592027689
R^2: 0.10246921830642874


In [36]:
power = 2
X = train_set[['stp', 'precipitation' ,'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y = train_set['value']

X_poly = poly_process.fit_transform(X)

X_test = test_set[['stp', 'precipitation', 'wind_speed', 'max_sustained_wind', 'max_temp', 'min_temp','temperature', 'dewpoint']]
y_test = test_set['value']

X_poly_test = poly_process.fit_transform(X_test)

lrmodel = LinearRegression()
lrmodel.fit(X_poly ,y)

y_pred = lrmodel.predict(X_poly)
print(f'Results for linear regression on training data')
print('Internal Parameters:')
print(f'   Bias is {lrmodel.intercept_}')
print(f'   Coefficients: {lrmodel.coef_}')
print(f'   Score {lrmodel.score(X_poly,y)}')

print(f'MAE is {mean_absolute_error(y, y_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y, y_pred))}')
print(f'MSE is {mean_squared_error(y, y_pred)}')
print(f'R^2: {r2_score(y, y_pred)}')

Results for linear regression on training data
Internal Parameters:
   Bias is 67475.17299110847
   Coefficients: [ 3.52482779e-01 -1.37914861e+02  1.97316911e+06 -5.66065259e+03
  5.27520002e+03  1.21827558e+03  4.85712354e+03 -8.35765396e+03
  1.75566312e+03  6.46951665e-02 -4.10047247e+03  1.05332763e+01
 -1.01315324e+01 -3.11667345e+00 -1.01436616e+01  1.81452582e+01
 -3.51725200e+00  1.99925842e+04 -2.28241839e+04  2.07154409e+04
 -9.06990180e+03 -6.99396623e+03  8.62823811e+03  1.02922841e+04
  7.65679862e+01 -3.91337588e+01 -3.09174879e+01  1.02607934e+01
  3.76297158e+01 -6.70046079e+00  3.83192496e+00  3.24610054e+01
  2.72453258e+01 -6.86695526e+01  1.26119262e+00 -3.73665453e-02
 -2.54815295e+00  3.62080298e+00  6.16298671e+00  7.93746579e+00
 -3.73253076e+00 -8.78674187e+00 -4.90495819e+00 -5.50240875e-01
 -1.59762278e-01  5.61343040e-06  2.13307758e+00 -4.87460369e-03
  4.86520032e-03  1.90967350e-03  5.29200141e-03 -9.80049348e-03
  1.76357990e-03 -2.08071724e+01  2.46278

In [37]:
y_test_pred = lrmodel.predict(X_poly_test)
print()
print(f'Results for linear regression on test data')
print(f'    Polynomial regression with degree {power}')
print(f'MAE is {mean_absolute_error(y_test, y_test_pred)}')
print(f'RMSE is {np.sqrt(mean_squared_error(y_test, y_test_pred))}')
print(f'MSE is {mean_squared_error(y_test, y_test_pred)}')
print(f'R^2: {r2_score(y_test, y_test_pred)}')


Results for linear regression on test data
    Polynomial regression with degree 2
MAE is 65.31749670137104
RMSE is 528.8910847265206
MSE is 279725.7795031956
R^2: -7592.69165337089
